### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [65]:
print(movies.shape)
movies.head()

(32355, 3)


,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [66]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32355 entries, 0 to 32354
Data columns (total 3 columns):
movie_id    32355 non-null object
movie       32355 non-null object
genre       32126 non-null object
dtypes: object(3)
memory usage: 758.4+ KB


Some null values in "genre".

In [67]:
movies.genre.unique()

array(['Documentary|Short', nan, 'Short|Horror', ...,
       'Crime|Horror|Musical', 'Documentary|Crime|Thriller',
       'Documentary|Reality-TV'], dtype=object)

In [84]:
movies.genre.nunique()

1203

That number is not correct, because genres are internally separated by "|". They are not mutually excusive.

In [92]:
def gather_genres(movies):
    genre_lists = movies.genre.dropna().str.split('|')
    genre_set = {g for genre_list in genre_lists
                 for g in genre_list}
    return list(genre_set)

In [94]:
genres = gather_genres(movies)
print(len(genres))
print(genres)

28
['Horror', 'Film-Noir', 'History', 'Game-Show', 'Action', 'Sport', 'Mystery', 'Drama', 'Reality-TV', 'War', 'Musical', 'Animation', 'News', 'Comedy', 'Fantasy', 'Crime', 'Adult', 'Documentary', 'Thriller', 'Romance', 'Family', 'Biography', 'Music', 'Short', 'Sci-Fi', 'Western', 'Talk-Show', 'Adventure']


In [70]:
print(reviews.shape)
reviews.head()

(743234, 4)


,user_id,movie_id,rating,timestamp
0,1,0068646,10,1381620027
1,1,0113277,10,1379466669
2,2,0422720,8,1412178746
3,2,0454876,8,1394818630
4,2,0790636,7,1389963947


In [71]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743234 entries, 0 to 743233
Data columns (total 4 columns):
user_id      743234 non-null object
movie_id     743234 non-null object
rating       743234 non-null int64
timestamp    743234 non-null object
dtypes: int64(1), object(3)
memory usage: 22.7+ MB


In [72]:
reviews.describe()

,rating
count,743234.000000
mean,7.299805
std,1.856125
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [73]:
reviews.user_id.unique().shape

(55437,)

In [74]:
reviews.user_id.nunique()

55437

In [75]:
reviews.rating.isnull().sum()

0

In [76]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 32355,
'The number of ratings in the dataset': 743234,
'The number of different genres': 1203,
'The number of unique users in the dataset': 55437,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.299805,
'The minimum rating given across all ratings': 0.0,
'The maximum rating given across all ratings': 10.0
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [77]:
print(movies.shape)
movies.head()

(32355, 3)


,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [78]:
import re

In [79]:
movies.movie.str.extract('.*(\d{4})').shape

(32355, 1)

In [80]:
movies.movie.str.extract('.*(\d{4})').isnull().sum()

0    0
dtype: int64

In [81]:
movies['date'] = movies.movie.str.extract('.*(\d{4})').astype(int)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [82]:
(movies.date < 1800).sum()

0

In [98]:
centuries = [1800, 1900, 2000]
for c in centuries:
    end = c + 100
    movies['{}\'s'.format(c)] = ((movies.date >= c) & (movies.date < end)).astype(int)
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [101]:
def dummy_genres(movies):
    genres = gather_genres(movies)
    for genre in genres:
        movies[genre] = movies.genre.str.contains(genre, na=False).astype(int)
    return movies

In [103]:
cleaned_movies = dummy_genres(movies)

In [130]:
cleaned_movies.movie_id = cleaned_movies.movie_id.astype(int)
cleaned_movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Horror,Film-Noir,History,...,Thriller,Romance,Family,Biography,Music,Short,Sci-Fi,Western,Talk-Show,Adventure
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [135]:
reviews['date'] = reviews.timestamp.apply(lambda x: pd.Timestamp.fromtimestamp(int(x)))
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0068646,10,1381620027,2013-10-12 21:20:27
1,1,0113277,10,1379466669,2013-09-17 22:11:09
2,2,0422720,8,1412178746,2014-10-01 12:52:26
3,2,0454876,8,1394818630,2014-03-14 14:37:10
4,2,0790636,7,1389963947,2014-01-17 11:05:47


In [136]:
reviews.movie_id = reviews.movie_id.astype(int)

In [137]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,68646,10,1381620027,2013-10-12 21:20:27
1,1,113277,10,1379466669,2013-09-17 22:11:09
2,2,422720,8,1412178746,2014-10-01 12:52:26
3,2,454876,8,1394818630,2014-03-14 14:37:10
4,2,790636,7,1389963947,2014-01-17 11:05:47


In [104]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year

In [105]:
print(movies_new.shape)
movies_new.head()

(31245, 36)


,Unnamed: 0,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [106]:
print(reviews_new.shape)
reviews_new.head()

(712337, 24)


,Unnamed: 0,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [117]:
movies_n = movies_new.set_index('movie_id').drop('Unnamed: 0', axis=1)
print(movies_n.shape)
movies_n.head()

(31245, 34)


,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [119]:
c_movies = cleaned_movies.set_index('movie_id')
print(c_movies.shape)
c_movies.head()

(32355, 34)


,movie,genre,date,1800's,1900's,2000's,Horror,Film-Noir,History,Game-Show,...,Thriller,Romance,Family,Biography,Music,Short,Sci-Fi,Western,Talk-Show,Adventure
movie_id,,,,,,,,,,,,,,,,,,,,,
0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [129]:
movies_n

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
439,The Great Train Robbery (1903),Short|Action|Crime,1903,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
443,"Hiawatha, the Messiah of the Ojibway (1903)",NaN,1903,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
628,The Adventures of Dollie (1908),Action|Short,1908,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
